### Imports

In [31]:
import torch
import classiq
from tqdm.auto import tqdm

import torch.nn as nn
import torch.optim as optim
from torchvision import datasets
from torch.utils.data import DataLoader
import torchvision.transforms as transforms

from classiq import create_model, synthesize, show, QFunc, QArray, QBit, Output, allocate, RX, RY, RZ, RZZ, RXX, RYY, CZ
from classiq.applications.qnn import QLayer
from classiq.execution import execute_qnn
from classiq.synthesis import SerializedQuantumProgram

from classiq.applications.qnn.types import (
    MultipleArguments,
    SavedResult,
    ResultsCollection,
)

classiq.authenticate()

In [32]:
device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cpu'

### Preparing Data

#### Getting a Dataset

In [33]:
def input_transform(image):
    """
    The input MNIST images are all 28 × 28. This function will firstly center-crop 
    them to 24 × 24 and then down-sample them to 4 × 4 for MNIST. Then we convert 
    the image pixels into angles for passing them into Rotation gates later for encoding.
    """
    image = transforms.ToTensor()(image)
    image = transforms.CenterCrop(24)(image)
    image = transforms.Resize(size = (4,4))(image)
    image = image.squeeze()
    image_pixels = torch.flatten(image)
    angles = torch.sqrt(image_pixels / 256)
    
    return angles

In [34]:
def target_transform(x):
    return x

In [35]:
# Setup training data
train_data = datasets.MNIST(
    root="data",
    train=True,
    download=True,
    transform=input_transform,
    target_transform=target_transform
)

test_data = datasets.MNIST(
    root="data",
    train=False,
    download=True,
    transform=input_transform,
    target_transform=target_transform
)

#### Visualize the Data

In [36]:
len(train_data), len(test_data)

(60000, 10000)

In [37]:
# See the first training example
image, label = train_data[0]
image, label

(tensor([0.0000, 0.0000, 0.0317, 0.0378, 0.0000, 0.0336, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0477, 0.0000, 0.0295, 0.0620, 0.0000, 0.0000]),
 5)

#### Prepare Dataloader

In [38]:
from torch.utils.data import Subset

# Define the size of the subset
subset_size = 64

# Create subsets of the datasets
train_subset = Subset(train_data, range(subset_size))
test_subset = Subset(test_data, range(subset_size))

In [39]:
# Setup the batch size hyperparameter
BATCH_SIZE = 32

# Turn datasets into iterables (batches)
train_dataloader = DataLoader(train_subset,
    batch_size=BATCH_SIZE, 
    shuffle=True
)

test_dataloader = DataLoader(test_subset,
    batch_size=BATCH_SIZE,
    shuffle=False
)

In [40]:
# Let's check out what we've created
print(f"Dataloaders: {train_dataloader, test_dataloader}") 
print(f"Length of train dataloader: {len(train_dataloader)} batches of {BATCH_SIZE}")
print(f"Length of test dataloader: {len(test_dataloader)} batches of {BATCH_SIZE}")

Dataloaders: (<torch.utils.data.dataloader.DataLoader object at 0x7f46c3f731d0>, <torch.utils.data.dataloader.DataLoader object at 0x7f46b79cd810>)
Length of train dataloader: 2 batches of 32
Length of test dataloader: 2 batches of 32


### Quantum Model

In [41]:
@QFunc
def encoding(q: QArray[QBit]) -> None:
    """
    This function encodes the input data into the qubits. This input data is a 4x4 image pixel values 
    converted into angle for rotation gates (RX, RY, RZ, RX) in form of a 16x1 vector. 
    We encode 4 pixels per qubit.

    Args:
        q (QArray[QBit]): Array of four Qubits to encode the input data into.
    """
    RX(theta="input_0", target=q[0]) # Pixel 0 on Qubit 0
    RY(theta="input_1", target=q[0]) # Pixel 1 on Qubit 0
    RZ(theta="input_2", target=q[0]) # Pixel 2 on Qubit 0
    RX(theta="input_3", target=q[0]) # Pixel 3 on Qubit 0
    
    RX(theta="input_4", target=q[1]) # Pixel 4 on Qubit 1
    RY(theta="input_5", target=q[1]) # Pixel 5 on Qubit 1
    RZ(theta="input_6", target=q[1]) # Pixel 6 on Qubit 1
    RX(theta="input_7", target=q[1]) # Pixel 7 on Qubit 1
    
    RX(theta="input_8", target=q[2]) # Pixel 8 on Qubit 2
    RY(theta="input_9", target=q[2]) # Pixel 9 on Qubit 2
    RZ(theta="input_10", target=q[2]) # Pixel 10 on Qubit 2
    RX(theta="input_11", target=q[2]) # Pixel 11 on Qubit 2
    
    RX(theta="input_12", target=q[3]) # Pixel 12 on Qubit 3
    RY(theta="input_13", target=q[3]) # Pixel 13 on Qubit 3
    RZ(theta="input_14", target=q[3]) # Pixel 14 on Qubit 3
    RX(theta="input_15", target=q[3]) # Pixel 15 on Qubit 3


In [42]:
@QFunc
def mixing(q: QArray[QBit]) -> None:
    """
    This function performs the mixing operation on the qubits. 
    This is done by applying a series of RZZ, RXX, RYY gates to form a
    ring connection.

    Args:
        q (QArray[QBit]): Array of four Qubits to apply the mixing operation on.
    """
    RZZ(theta="weight_0", target=q[0:2])
    RZZ(theta="weight_1", target=q[1:3])
    RZZ(theta="weight_2", target=q[2:4])
    
    RXX(theta="weight_4", target=q[0:2])
    RXX(theta="weight_5", target=q[1:3])
    RXX(theta="weight_6", target=q[2:4])
    
    RYY(theta="weight_8", target=q[0:2])
    RYY(theta="weight_9", target=q[1:3])
    RYY(theta="weight_10", target=q[2:4])

In [43]:
@QFunc
def cz_block(q: QArray[QBit]) -> None:
    """
    This function applies CZ gates between each qubit.

    Args:
        q (QArray[QBit]): Array of four Qubits to apply the entanglement operation on.
    """
    CZ(control=q[0], target=q[1])
    CZ(control=q[1], target=q[2])
    CZ(control=q[2], target=q[3])

In [44]:
@QFunc
def main(res: Output[QArray[QBit]]) -> None:
    """
    This is the main function from which model will be created. 
    It calls the other functions to perform the encoding, mixing and entanglement.

    Args:
        res (Output[QArray[QBit]]): Output QArray of QBits from which the model will be created.
    """
    allocate(4, res)
    encoding(q=res)
    mixing(q=res)
    cz_block(q=res)

In [45]:
# Create a model
model = create_model(main)
quantum_program = synthesize(model)
# show(quantum_program)

### Quantum Neural Network

In [46]:
def execute(quantum_program: SerializedQuantumProgram, arguments: MultipleArguments) -> ResultsCollection:
    return execute_qnn(quantum_program, arguments)

In [47]:
def post_process(result: SavedResult) -> torch.Tensor:
    counts: dict = result.value.counts
    # The probability of measuring |0>
    print(f"counts: {counts}")
    p_zero: float = counts.get("0", 0.0) / sum(counts.values())
    return torch.tensor(p_zero)

In [48]:
class Net(torch.nn.Module):
    def __init__(self, *args, **kwargs) -> None:
        super().__init__()
        self.qlayer = QLayer(
            quantum_program,
            execute,
            post_process,
            *args,
            **kwargs
        )
        
    def forward(self, x: torch.Tensor) -> torch.Tensor:
        x = self.qlayer(x)
        return x

In [49]:
qnn = Net()

### Training and Testing Loop

In [50]:
_LEARNING_RATE = 1.0

# choosing our loss function
loss_fn = nn.L1Loss()
# choosing our optimizer
optimizer = optim.SGD(qnn.parameters(), lr=_LEARNING_RATE)

In [51]:
def train(
    model: nn.Module,
    data_loader: DataLoader,
    loss_fn: nn.modules.loss._Loss,
    optimizer: optim.Optimizer,
    epochs: int = 20,
) -> None:
    for epoch in tqdm(range(epochs)):
        print(f"Epoch: {epoch}\n----------")
        for data, label in data_loader:
            optimizer.zero_grad()
            output = model(data)
            loss = loss_fn(output, label)
            loss.backward()
            optimizer.step()

In [52]:
type(train_dataloader)

torch.utils.data.dataloader.DataLoader

In [53]:
train(qnn, train_dataloader, loss_fn, optimizer)

  0%|          | 0/20 [00:00<?, ?it/s]

Epoch: 0
----------
counts: {'1010': 3, '0101': 1, '0110': 13, '1001': 1, '1000': 5, '0010': 3, '0000': 2022}
counts: {'0100': 1, '0011': 1, '1010': 4, '0110': 13, '1000': 4, '1001': 2, '0010': 8, '0000': 2015}
counts: {'0100': 1, '0001': 1, '1010': 3, '0101': 1, '0110': 13, '1001': 1, '1000': 5, '0010': 2, '0000': 2021}
counts: {'0001': 1, '0011': 1, '1010': 4, '0110': 18, '0000': 2024}
counts: {'0011': 1, '0001': 1, '1010': 3, '0101': 1, '0110': 12, '1001': 1, '1000': 6, '0010': 1, '0000': 2022}
counts: {'1010': 3, '0100': 1, '0110': 16, '1001': 1, '1000': 2, '0010': 3, '0000': 2022}
counts: {'0011': 1, '0100': 1, '1010': 4, '0110': 16, '1000': 2, '0000': 2024}
counts: {'0101': 1, '1010': 4, '0110': 12, '1000': 6, '0010': 1, '0000': 2024}
counts: {'0011': 1, '1010': 4, '0110': 18, '0000': 2025}
counts: {'0011': 1, '1010': 4, '0110': 18, '0000': 2025}
counts: {'0011': 1, '1010': 4, '0110': 18, '0000': 2025}
counts: {'0011': 1, '0001': 1, '0100': 1, '1010': 4, '0110': 18, '0010': 1, '0

  5%|▌         | 1/20 [01:50<35:02, 110.64s/it]

counts: {'0011': 1, '1010': 4, '0110': 18, '0000': 2025}
counts: {'0011': 1, '1010': 4, '0110': 18, '0000': 2025}
counts: {'0011': 1, '1010': 4, '0110': 18, '0000': 2025}
counts: {'0011': 1, '1010': 4, '0110': 18, '0000': 2025}
counts: {'0001': 1, '0011': 1, '1010': 4, '0110': 16, '1001': 2, '0000': 2024}
counts: {'1010': 3, '0011': 1, '1001': 1, '0110': 18, '0000': 2025}
counts: {'0011': 1, '1010': 4, '0110': 18, '0000': 2025}
counts: {'0011': 1, '1010': 4, '0110': 18, '0000': 2025}
counts: {'0011': 1, '1010': 4, '0110': 18, '0000': 2025}
counts: {'0001': 1, '0011': 1, '1010': 4, '0110': 18, '0000': 2024}
counts: {'0101': 1, '1010': 4, '0110': 16, '1001': 2, '0010': 1, '0000': 2024}
counts: {'1010': 3, '0101': 2, '0001': 1, '1001': 1, '0110': 16, '0000': 2025}
counts: {'0011': 1, '1010': 4, '0110': 18, '0000': 2025}
counts: {'0001': 1, '0011': 1, '1010': 4, '0110': 18, '0000': 2024}
counts: {'0001': 1, '0011': 1, '1010': 4, '0110': 18, '0000': 2024}
counts: {'0011': 1, '1010': 4, '011

 10%|█         | 2/20 [02:57<25:32, 85.15s/it] 

...

counts: {'0100': 1, '1010': 4, '0110': 16, '1000': 2, '0010': 1, '0000': 2024}
counts: {'0100': 1, '1010': 4, '0110': 16, '1000': 2, '0010': 1, '0000': 2024}
counts: {'0100': 1, '1010': 4, '0110': 16, '1000': 2, '0010': 1, '0000': 2024}
counts: {'0100': 1, '1010': 4, '0110': 16, '1000': 2, '0010': 1, '0000': 2024}
counts: {'0001': 2, '0101': 1, '1010': 4, '0110': 16, '1001': 2, '0010': 1, '0000': 2022}
counts: {'1010': 3, '0011': 1, '1001': 1, '0110': 18, '0010': 1, '0000': 2024}
counts: {'0100': 1, '1010': 4, '0110': 16, '1000': 2, '0010': 1, '0000': 2024}
counts: {'0100': 1, '1010': 4, '0110': 16, '1000': 2, '0010': 1, '0000': 2024}
counts: {'0100': 1, '1010': 4, '0110': 16, '1000': 2, '0010': 1, '0000': 2024}
counts: {'0001': 2, '0011': 1, '1010': 4, '0110': 16, '1000': 2, '0010': 1, '0000': 2022}
counts: {'0011': 1, '0101': 1, '1010': 4, '0110': 15, '1000': 1, '1001': 2, '0010': 2, '0000': 2022}
counts: {'0001': 1, '1010': 3, '0101': 2, '1000': 1, '0110': 16, '0010': 1, '0000': 202

 95%|█████████▌| 19/20 [25:23<01:10, 70.37s/it]

counts: {'0011': 1, '1010': 4, '0110': 18, '0000': 2025}
counts: {'0011': 1, '1010': 4, '0110': 18, '0000': 2025}
counts: {'0011': 1, '1010': 4, '0110': 18, '0000': 2025}
counts: {'0011': 1, '1010': 4, '0110': 18, '0000': 2025}
counts: {'0011': 1, '1010': 4, '0110': 16, '1001': 2, '0000': 2025}
counts: {'1010': 3, '0011': 1, '1001': 1, '0110': 18, '0000': 2025}
counts: {'0011': 1, '1010': 4, '0110': 18, '0000': 2025}
counts: {'0011': 1, '1010': 4, '0110': 18, '0000': 2025}
counts: {'0011': 1, '1010': 4, '0110': 18, '0000': 2025}
counts: {'0011': 1, '1010': 4, '0110': 18, '0000': 2025}
counts: {'0101': 1, '1010': 4, '0110': 16, '1001': 2, '0000': 2025}
counts: {'1010': 3, '0101': 2, '1001': 1, '0110': 16, '0000': 2026}
counts: {'0011': 1, '1010': 4, '0110': 18, '0000': 2025}
counts: {'0011': 1, '1010': 4, '0110': 18, '0000': 2025}
counts: {'0011': 1, '1010': 4, '0110': 18, '0000': 2025}
counts: {'1010': 4, '0110': 18, '0000': 2026}
counts: {'0101': 2, '1010': 4, '0110': 16, '0000': 2026

100%|██████████| 20/20 [26:35<00:00, 79.79s/it]

counts: {'0001': 2, '0101': 1, '1010': 4, '0110': 12, '1000': 6, '0010': 1, '0000': 2022}
counts: {'0001': 2, '0101': 1, '1010': 4, '0110': 12, '1000': 6, '0010': 1, '0000': 2022}
counts: {'0001': 2, '0101': 1, '1010': 4, '0110': 12, '1000': 6, '0010': 1, '0000': 2022}
counts: {'0001': 2, '0101': 1, '1010': 4, '0110': 12, '1000': 6, '0010': 1, '0000': 2022}
counts: {'0001': 2, '0101': 1, '1010': 4, '0110': 12, '1000': 4, '1001': 2, '0010': 1, '0000': 2022}
counts: {'0001': 2, '1010': 3, '0101': 1, '0110': 14, '1001': 1, '1000': 4, '0010': 1, '0000': 2022}
counts: {'0001': 2, '0101': 1, '1010': 4, '0110': 12, '1000': 6, '0010': 1, '0000': 2022}
counts: {'0001': 2, '0101': 1, '1010': 4, '0110': 12, '1000': 6, '0010': 1, '0000': 2022}
counts: {'0001': 2, '0101': 1, '1010': 4, '0110': 12, '1000': 6, '0010': 1, '0000': 2022}
counts: {'0001': 2, '0101': 1, '1010': 4, '0110': 12, '1000': 6, '0010': 1, '0000': 2022}
counts: {'0101': 1, '0001': 1, '1010': 4, '0110': 13, '1000': 4, '1001': 2, '0

In [59]:
def test(
    model: nn.Module,
    data_loader: DataLoader,
    atol=1e-4
) -> float:
    num_correct = 0
    total = 0
    
    # Put the model in eval mode
    model.eval()
    
    # Turn on inference mode context manager
    with torch.inference_mode():
        for data, labels in data_loader:
            # Let the model predict
            predictions = model(data)
            
            # Get a tensor of booleans, indicating if each label is close to the real label
            is_prediction_correct = predictions.isclose(labels, atol=atol)
            
            # Count the amount of `True` predictions
            num_correct += is_prediction_correct.sum().item()
            
            # Count the total evaluations
            #   the first dimension of `labels` is `batch_size`
            total += labels.size(0)
    
    # Calculate the accuracy
    accuracy = float(num_correct) / float(total)
    print(f"Test Accuracy of the model: {accuracy*100:.2f}")
    return accuracy

In [ ]:
test(qnn, test_dataloader)